# 01 element perturbation

Within the following tutorial you are going to see how to perturb one element of the GeneralGraph.

### Input description

The input for the graph construction currently consists of text files reflecting the hierarchy of the plant elements and their features. In the text input files each line corresponds to a node/element description.
The same line reports the name of the predecessor of a particular node/element, the relationship between them, and the list of node’s attributes (area in which the element is 
present, perturbation resistance, etc.).
In this way each line corresponds to an edge connecting a element to its parent element.

Each line should contain the following info:
- element id (**Mark**)
- parent of the element id (**father_mark**)
- parent-child relationship 
(**father_cond**: *AND*, *OR*, *SINGLE*, *ORPHAN*. It is an edge attribute.)
- type of element 
(**Description**: *isolation_A*, *isolation_B* are isolating elements 
with opposite behaviour. It is a node attribute.)
- state of the isolating element 
(**InitStatus**: *1*, *0*. It is a node attribute.)
- area in which the element is located 
(**Area**. It is a node attribute.)
- element external perturbation resistance 
(**PerturbationResistant**: *1*, *0*. It is a node attribute.)
- source - hub - user elements 
(**Type**: *SOURCE* or *HUB* or *USER*. It is a node attribute.)

The hierarchy of the elements explains how commodities flow from one element to another element and from one system to another system. In fact, if the input is properly formatted, with this simple digraph model it is possible to represent and integrate different interconnected plants in a unique graph without losing information about their peculiarities. 

In the graph, the nodes represent the system elements while the edges connecting the nodes harbor the logic relations (edge attributes) existing between the elements (*ORPHAN*, *SINGLE*, *AND*, and *OR*):
- An **ORPHAN** edge is the edge of a node without predecessors.
- A **SINGLE** edge connects a node to its only one predecessor.
- An **AND** edge indicates that the node/element has more than one predecessor. All the predecessors are necessary for the functioning of that element.
- An **OR** edge indicates that the node/element has more than one predecessor. Just one of the node’s predecessors should be active to guarantee the functioning of the element. For this reason, correct input formatting is one of the most important steps of the analysis.

### Example

In the cartoon below it is represented the example input file `TOY_graph.csv`.
In this file are present 19 nodes/elements connected by direct edges that reflect the hierarchy of the system in a parent-child fashion.

The nodes are distributed in adjacent areas.

In area1 are present 5 nodes: 1, 2, 3, 4 and 5.

In area2 are present nodes: 11, 19, 12, 13, 14 and 18.

In area3 are present nodes: 15, 9, 16, 17 and 10.

In area4 are present nodes: 6, 7 and 8.

A perturbation of one or multiple elements in one area may exceed the area boundaries and propagate to other systems connected to it, located in other areas. 

Nodes 2, 3, 4, 5 are perturbation resistant nodes ("PerturbationResistant" field = "1").
These nodes will not be affected by the simulated perturbation.

Nodes 2 and 3 are isolating elements ("Description field" = "isolation_A"). In the figure, perturbing node 1 would result in the breakage of all the nodes present in the graph except 
node 15 in absence of isolating elements. On the other hand, isolating elements 2 and 3 would stop the perturbation propagation cascade to node 1.

<img src="./input_files/TOY_graph.png" alt="TOY_graph" width="800" height="600"/>

### Simulate a perturbation to element "1"

Let us see how to generate a perturbation that propagates from element "1" of the TOY graph. First of all, we need to import GRAPE GeneralGraph. We are also importing `pandas` package since we are going to use it for reading the `CSV` output files obtained.

In [1]:
import pandas as pd
from grape.general_graph import GeneralGraph

Second of all, we define a GeneralGraph and we load the nodes for the TOY graph from the input file.

In [2]:
g = GeneralGraph()
g.load("./input_files/TOY_graph.csv")

Let us check the input before perturbing the graph in any way. In order to do so we use `check_input_with_gephi` function, which prints out the list of edges and the list of nodes of the current graph.  The two output files generated can be used to visualize the input with Gephi.

In [3]:
g.check_input_with_gephi()

Let us have a look at the list of nodes and edges that get printed for the integer TOY graph.

In [4]:
nodes = pd.read_csv("check_import_nodes.csv", na_filter=False)
nodes

,Mark,Description,InitStatus,PerturbationResistant,Area
0,1,,,0,area1
1,2,isolation_A,1,1,area1
2,3,isolation_A,1,1,area1
3,4,,,1,area1
4,5,,,1,area1
5,6,,,0,area4
6,7,,,0,area4
7,8,,,0,area4
8,9,,,0,area3
9,15,,,0,area3


In [5]:
edges = pd.read_csv("check_import_edges.csv", na_filter=False)
edges

,Mark,Father_mark
0,2,1
1,3,1
2,4,2
3,5,3
4,6,4
5,6,7
6,7,6
7,8,6
8,6,8
9,9,8


In order to perturb an element, we are going to use the function `simulate_element_perturbation`, which takes as an argument the list of the nodes that we want to damage.

In [6]:
g.simulate_element_perturbation(["1"])

PROC NUM 4
In the graph are present 19 nodes
go serial!
the graph is dense, density = 0.07894736842105263
PROC NUM 4
In the graph are present 18 nodes
go serial!
the graph is dense, density = 0.08169934640522876


The execution of the element perturbation gives us information about what is going on (`PROC NUM`, the number of processors used may be different in your case).
From the output of the command we see that the serial version of the software has been used. This depends on the computational load requested by the particular graph: after a certain size of the graph, the parallel version is automatically called. However, our toy graph containing 19 nodes is not large, and the serial version is enough. You can check the threshold for switching to parallel in the source code.
The size of the integer graph is reported (19 nodes), and depending on the graph density different methods for shortest path calculation are adopted. In particular, for dense graphs like the one in this tutorial, Floyd-Warshall algorithm is adopted.

### Output

The simulation of the damage generates a few output files, which give us a diagnostics of the fault that just happened. Before going through them, anyway, let us have a look at the logging debug output contained into the file `general_code_output.log`.

In [7]:
logfile = open("general_code_output.log")
print(logfile.read())
logfile.close()

DEBUG:root:Visited: {'1'}
DEBUG:root:Node: 1
DEBUG:root:Predecessors: []
DEBUG:root:Broken: ['1']
DEBUG:root:Visited: {'2', '1'}
DEBUG:root:Node: 2
DEBUG:root:Node 2 visited, fault resistant node
DEBUG:root:Visited: {'2', '3', '1'}
DEBUG:root:Node: 3
DEBUG:root:Node 3 visited, fault resistant node



The file for debug gives us an idea of what happens when a node is perturbed. Node 1 has no fault resistance of any kind; hence, it gets broken. After that, the perturbation gets propagated on all neighboring nodes, namely nodes 2 and 3 in our case. They are visited, but not affected by the damage, being fault resistant.

Output file `element_perturbation.csv` lists the new status of the elements the new status of the elements (active, not-active) and the areas in which the elements are located (affected, not affected) as well as the new status of elements that have been operated to stop the propagation of the perturbation or to open new paths in case of unavailabilty of the default ones. 
The efficiency (*nodal efficiency*, *local efficiency*, *global efficiency*) and closeness indices (*closeness_centrality*, *betweenness_centrality*, *indegree_centrality*) are then recalculated.

In [8]:
elements = pd.read_csv("element_perturbation.csv", na_filter=False)
elements

,Mark,Description,InitStatus,IntermediateStatus,FinalStatus,Mark_Status,PerturbationResistant,Area,Status_Area,closeness_centrality,...,indegree_centrality,outdegree_centrality,original_local_eff,final_local_eff,original_nodal_eff,final_nodal_eff,original_avg_global_eff,final_avg_global_eff,original_service,final_service
0,1,,,,,NOT_ACTIVE,0,area1,DAMAGED,0.000000,...,0.000000,0.111111,0.174374,,0.321362,,0.175919,,0.0,
1,2,isolation_A,1,,,ACTIVE,1,area1,DAMAGED,0.055556,...,0.055556,0.055556,0.202227,0.21412231559290384,0.196896,0.20847763347763348,0.175919,0.17771187599618973,0.0,0.0
2,3,isolation_A,1,,,ACTIVE,1,area1,DAMAGED,0.055556,...,0.055556,0.055556,0.148148,0.1568627450980392,0.151852,0.1607843137254902,0.175919,0.17771187599618973,0.0,0.0
3,4,,,,,ACTIVE,1,area1,DAMAGED,0.074074,...,0.055556,0.055556,0.225838,0.2391223155929038,0.202227,0.21412231559290384,0.175919,0.17771187599618973,0.0,0.0
4,5,,,,,ACTIVE,1,area1,DAMAGED,0.074074,...,0.055556,0.055556,0.148148,0.15686274509803924,0.148148,0.1568627450980392,0.175919,0.17771187599618973,0.0,0.0
5,6,,,,,ACTIVE,0,area4,AVAILABLE,0.173611,...,0.166667,0.111111,0.211828,0.22428804855275447,0.225838,0.2391223155929038,0.175919,0.17771187599618973,0.0,0.0
6,7,,,,,ACTIVE,0,area4,AVAILABLE,0.115741,...,0.055556,0.055556,0.225838,0.2391223155929038,0.174449,0.18471055088702149,0.175919,0.17771187599618973,0.0,0.0
7,8,,,,,ACTIVE,0,area4,AVAILABLE,0.115741,...,0.055556,0.111111,0.193541,0.2049253034547152,0.249206,0.26386554621848746,0.175919,0.17771187599618973,0.0,0.0
8,9,,,,,ACTIVE,0,area3,AVAILABLE,0.143275,...,0.111111,0.055556,0.156481,0.16568627450980392,0.161243,0.1707282913165266,0.175919,0.17771187599618973,0.0,0.0
9,15,,,,,ACTIVE,0,area3,AVAILABLE,0.000000,...,0.000000,0.055556,0.161243,0.1707282913165266,0.165873,0.17563025210084032,0.175919,0.17771187599618973,0.0,0.0


There is a lot of information contained in this output file. We would like to focus on some.
The status of the nodes contained in `Mark_Status` and `Status_Area` reflect the consequences of the damage: broken nodes are defined as `NOT_ACTIVE`, while areas that contain broken nodes are labeled as `DAMAGED`.

In [9]:
elements[['Mark_Status', 'Status_Area']]

,Mark_Status,Status_Area
0,NOT_ACTIVE,DAMAGED
1,ACTIVE,DAMAGED
2,ACTIVE,DAMAGED
3,ACTIVE,DAMAGED
4,ACTIVE,DAMAGED
5,ACTIVE,AVAILABLE
6,ACTIVE,AVAILABLE
7,ACTIVE,AVAILABLE
8,ACTIVE,AVAILABLE
9,ACTIVE,AVAILABLE


Moreover, the nodes in the GeneralGraph are labeled as *SOURCE*, *HUB* or *USER*, and commodities flow from *SOURCE* to *USER*, passing through *HUB* nodes. In order to better understand this concept, let us load another output file generated by the perturbation simulation, `service_paths_element_perturbation.csv`, which reports the paths (all paths and shortest paths), if any, that connect *SOURCE* and *USER* elements (paths are calculated before and after perturbing one or multiple elements).

In [10]:
paths = pd.read_csv("service_paths_element_perturbation.csv", na_filter=False)
paths

,from,to,original_shortest_path_length,original_shortest_path,original_simple_path,original_pair_efficiency,ids,area,final_shortest_path_length,final_shortest_path,final_simple_path,final_pair_efficiency
0,1,18,6.0,"['1', '3', '5', '11', '19', '14', '18']","[['1', '2', '4', '6', '8', '9', '16', '17', '1...",0.166667,118,,,,,
1,15,18,8.0,"['15', '9', '16', '17', '10', '11', '19', '14'...","[['15', '9', '16', '17', '10', '11', '19', '12...",0.125000,1518,area3,8.0,"['15', '9', '16', '17', '10', '11', '19', '14'...","[['15', '9', '16', '17', '10', '11', '19', '12...",0.125


Since node 1 is broken, there is no final path between node 1 and node 18, while it still exists between nodes 15 and 18. Due to the damage of node 1, we can also see that the total final service at the *USER* 18 is smaller than the original one, lowered by 1 which is the service granted by the *SOURCE* represented by node 1, which is no more available.

In [11]:
elements[['original_service', 'final_service']].loc[18]

original_service      3
final_service       2.0
Name: 18, dtype: object